## Библиотеки

In [ ]:
import requests
import time
import pickle
import datetime
import json
import time
import pandas as pd
import os
import cv2
import numpy as np
from PIL import Image
from torchvision import models, transforms
from transformers import pipeline
import easyocr
from matplotlib import pyplot as plt
import seaborn as sns
import re

## Анализ изображений

In [ ]:
# Наличие текста
def detect_text(image_path):
    reader = easyocr.Reader(['en'])
    result = reader.readtext(image_path)
    return 1 if result else 0

# Оценка красочности
def calculate_colorfulness(image_path):
    image = cv2.imread(image_path)
    (B, G, R) = cv2.split(image.astype("float"))
    rg = np.abs(R - G)
    yb = np.abs(0.5 * (R + G) - B)
    rg_mean, rg_std = np.mean(rg), np.std(rg)
    yb_mean, yb_std = np.mean(yb), np.std(yb)
    std_root = np.sqrt((rg_std ** 2) + (yb_std ** 2))
    mean_root = np.sqrt((rg_mean ** 2) + (yb_mean ** 2))
    colorfulness = std_root + (0.3 * mean_root)
    return min(10, max(1, int(colorfulness / 10)))

## Анализ текста

In [ ]:
# Запрос к llama_request
def llama_request(prompt):
    """Отправка запроса к LLM и получение ответа."""
    # URL вашего сервера
    url = 'http://localhost:11434/api/chat'
    payload = {
        "model": "llama3.1:8b",
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "stream": False,
        "max_token": 10000
    }

    # Отправка POST-запроса
    response = requests.post(
        url,
        data=json.dumps(payload)
    )
    return response.json()['message']['content']

# Анализ текста с помощью llama
def analyze_post(post_text):
    # 1. Тематика поста
    topic_prompt = f"Определи основную тему текста: \"{post_text}\". Ответь только одним словом без точки"
    topics = llama_request(topic_prompt)
    
    # 2. Структура текста
    word_count = len(post_text.split())
    char_count = len(post_text)
    
    # Проверка наличия заголовков и списков
    headers = len(re.findall(r'(^|\n)(#+\s|\s*\d+\.\s|\*\s)', post_text))
    paragraphs = post_text.strip().split("\n\n")
    has_lists = bool(re.search(r'(^|\n)(\d+\.\s|\*\s)', post_text))
    
    # 3. Язык и стиль
    tone_prompt = f"Определи тональность текста одним словом (позитивная, негативная, нейтральная). Ответ только слово - тональность.: \"{post_text}\"."
    tone = llama_request(tone_prompt)
    
    slang_prompt = f"Оцени простоту текста ответив только оценку 1 до 10 без точки в конце. Оцени насколько сложен текст с точки зрения трудных слов, сложных/длинных граматических констркуций и тому подобное. \"{post_text}\"."
    uses_slang = llama_request(slang_prompt)
    
    emotion_prompt = f"Определи эмоциональную окраску текста одним словом: \"{post_text}\". Пример ответа: радостная, грустная, нейтральная. Ответь только слово - эмоциональная окраска. Больше ничего не пиши."
    emotion = llama_request(emotion_prompt)
    
    # Результаты анализа
    result = {
        "Тематика": topics,
        "Структура текста": {
            "Количество слов": word_count,
            "Количество символов": char_count,
            "Количество заголовков": headers,
            "Количество абзацев": len(paragraphs),
            "Есть списки": has_lists,
        },
        "Язык и стиль": {
            "Тональность": tone,
            "Сложность текста": uses_slang,
            "Эмоциональная окраска": emotion,
        },
    }
    
    return result

## Сбор данных

In [ ]:
access_token = '7467ea587467ea587467ea58a97776aa9e774677467ea58173abbe1a90fd6c976a91670'
owner_id = '-23242408'
# owner_id2 = 68661738
count = 0
offset = 0

img = "/Users/aleksandrcuvpilo/Desktop/Programming/vscode/python_folder/project/vkarminey/downloaded_image.jpg"


f = open("file.pkl", "wb")
with open("yes.txt", "r") as file:
    files = file.read()
    emoji = files.split()
temp = open('temp.txt', "wb")
def getjson(url, data=None):
    response = requests.get(url, params=data)
    response = response.json()
    return response

def get_all_postss(access_token, owner_id, count=500, offset=0):
    all_posts = []
    k = 0
    last_post_date = -1

    while True:
        time.sleep(0.9)
        wall = getjson('https://api.vk.com/method/wall.get',
                        {'owner_id': owner_id, 'offset': offset, 'count': count, 'access_token': access_token, 'v': '5.131'})
        count_posts = wall['response']['count']
        posts = wall['response']['items']

        all_posts.extend(posts)

        last_post_date = int(datetime.datetime.fromtimestamp(int(all_posts[-1]['date'])).strftime('%Y'))

        # if k % 50 == 0:
        #     print(k)
        # k += 1
        if last_post_date == 2023:
            break
        else:
            offset += 100
    return count_posts

def get_all_posts(access_token, owner_id, count=500, offset=0):
    all_posts = []
    k = 0
    last_post_date = -1

    while True:
        time.sleep(0.9)
        wall = getjson('https://api.vk.com/method/wall.get',
                        {'owner_id': owner_id, 'offset': offset, 'count': count, 'access_token': access_token, 'v': '5.131'})
        count_posts = wall['response']['count']
        posts = wall['response']['items']

        all_posts.extend(posts)

        last_post_date = int(datetime.datetime.fromtimestamp(int(all_posts[-1]['date'])).strftime('%Y'))

        # if k % 50 == 0:
        #     print(k)
        # k += 1
        if last_post_date == 2023:
            break
        else:
            offset += 100
    return all_posts

def filter_data(all_posts):
    filtered_data = []
    photo_cnt = 0
    for post in all_posts:
        try:
            id = post['id']
        except:
            id = ' '
        try:
            text = post['text']
        except:
            text = 'текста нет'
        try:
            likes = post['likes']['count']
        except:
            likes = 0
        try:
            date = post['date']
        except:
            date = 0
        try:
            cnt_comm = len(post['comments'])
        except:
            cnt_comm = 0
        photo_ids = []
        for i in post['attachments']:
            if i['type'] == 'photo':
                #print(i['photo'])
                #width = i['photo']['width']
                #height = i['photo']['height']
                #print(i['photo']['sizes'])
                for j in i['photo']['sizes']:
                    if j['type'] == 'x':
                        photo_cnt = 1
                        photo_ids.append(j['url'])
                        break
                if(photo_cnt == 0):
                    photo_ids.append(i['photo']['sizes'][0]['url'])

        filtered_post = {'id': id, 'text': text, 'likes': likes, 'date': date, 'cnt_comm': cnt_comm, 'photo_ids': photo_ids}

        filtered_data.append(filtered_post)

    return filtered_data

def get_all_liked_lists(access_token, owner_id, liked_object_id, count=1000, offset=0, friends_only=0):
    time.sleep(0.5)
    api_query = getjson('https://api.vk.com/method/likes.getList',
                        {'access_token': access_token, 'type': 'post', 'owner_id': owner_id, 'item_id': liked_object_id,
                            'filter': 'likes', 'friends_only': friends_only, 'count': count, 'v': '5.131'})
    Users_count = api_query['response']['count']
    List_of_users = api_query['response']['items']
    return Users_count, List_of_users

def get_all_users_bdate(access_token, user_ids, fields, count=1000, offset=0, friends_only=0):
    time.sleep(0.5)
    api_query_user_info = getjson('https://api.vk.com/method/users.get',
                                    {'access_token': access_token, 'user_ids': user_id, 'fields': 'bdate, sex', 'count': count,
                                    'v': '5.131'})
    User_Birth_date = api_query_user_info['response']
    # User_Birth_date = api_query_user_info['response'][0]
    return User_Birth_date

def get_subs(access_token, owner_id, count=100, offset=0):
    listo = []
    time.sleep(1)
    subs = getjson('https://api.vk.com/method/groups.getMembers',
                    {'group_id': owner_id, 'sort': 'time_asc', 'offset': offset, 'count': count, 'fields': 'bdate', 'access_token': access_token, 'v': '5.131'})
    csus = subs['response']['count']
    for i in range(csus):
        time.sleep(1)
        subs = getjson('https://api.vk.com/method/groups.getMembers',
                        {'group_id': owner_id, 'sort': 'time_asc', 'offset': offset, 'count': count, 'fields': 'bdate',
                        'access_token': access_token, 'v': '5.131'})
        csubs = subs['response']['items']
        listo.extend(csubs)
    return listo

def anylyse_url(url):
    response = requests.get(url)
    if response.status_code == 200:
        with open(img, "wb") as file:
            file.write(response.content)
        #print(f"Image downloaded successfully and saved as '{img}'.")
    else:
        pass
        #print(f"Failed to download image. HTTP status code: {response.status_code}")
    image_path = img
    has_text = detect_text(image_path)
    #has_people = detect_people(image_path)
    #objects = detect_objects(image_path)
    colorfulness = calculate_colorfulness(image_path)
    os.remove(img)
    return has_text, colorfulness

count = 0
offset = 0
main_posts = get_all_postss(access_token, owner_id)
if main_posts > 1000:
    main_posts = 999

all_posts = get_all_posts(access_token, owner_id)
final_filter = filter_data(all_posts)
ID_list = []
txt_list = []
likes_list = []
date_list = []
cnt_comm_list = []
photo_ids_list = []
for post1 in final_filter:
    ID_list.append(post1['id'])
    txt_list.append(post1['text'])
    likes_list.append(post1['likes'])
    date_list.append(post1['date'])
    cnt_comm_list.append(post1['cnt_comm'])
    photo_ids_list.append(post1['photo_ids'])

User_lists_collection = []
Final_list = {}
t = 0
for item in ID_list:
    text_id = txt_list[t]
    tsplit = text_id.split()
    emojicount = 0
    tagscount = 0
    wordcount = 0
    linkcount = 0
    for i in range(len(tsplit)):
        tsp = tsplit[i]
        x = tsp.find('#')
        if x != (-1):
            tagscount += 1
            wordcount -= 1
        for j in range(len(tsp)):
            if tsp[j] in emoji:
                emojicount +=1
        if tsp not in emoji:
            wordcount += 1
        y = tsp.find('.')
        if y != (-1) and y != len(tsp)-1:
            linkcount += 1
            wordcount -= 1
    liked_object_id = item
    User_list_of_responses = get_all_liked_lists(access_token, owner_id, liked_object_id)
    text_analyze = analyze_post(text_id)
    all_text = 0
    all_people = 0
    all_objects = []
    all_colorfulness = 0
    cnt_text = 0
    cnt_people = 0
    cnt_colorfulness = 0
    for j in photo_ids_list[t]:
        has_text, colorfulness = anylyse_url(j)
        all_text += int(has_text)
        #all_people += int(has_people)
        #all_objects.extend(objects)
        all_colorfulness += int(colorfulness)
        cnt_text += 1
        cnt_colorfulness += 1
    try:
        medium_photo_text = all_text/cnt_text
    except:
        medium_photo_text = np.nan
    try:
        medium_photo_colorfulness = all_colorfulness/cnt_colorfulness
    except:
        medium_photo_colorfulness = np.nan

    list_of_users_ids = User_list_of_responses[1]
    user_id = str(list_of_users_ids).strip('[]')
    person_bdate = get_all_users_bdate(access_token, owner_id, user_id)

    nfl = [i for i in person_bdate]
    female_all = [i for i in nfl if 'sex' in i and i['sex'] == 1]
    male_all = [i for i in nfl if 'sex' in i and i['sex'] == 2]

    new_filtered_list = [i for i in person_bdate if 'bdate' in i and len(i['bdate']) > 6 and int(i['bdate'][-4:]) >= 2002 and int(i['bdate'][-4:]) <= 2010]
    female_school = [i for i in new_filtered_list if 'sex' in i and i['sex'] == 1]
    male_school = [i for i in new_filtered_list if 'sex' in i and i['sex'] == 2]

    nfl2 = [i for i in person_bdate if 'bdate' in i and len(i['bdate']) > 6 and int(i['bdate'][-4:]) <= 1993]
    female_eldery = [i for i in nfl2 if 'sex' in i and i['sex'] == 1]
    male_eldery = [i for i in nfl2 if 'sex' in i and i['sex'] == 2]

    f = 0
    list_of_likes = len(new_filtered_list)
    list_of_likes2 = len(nfl2)
    lol = len(nfl)
    if list_of_likes != 0:
        f = (list_of_likes / lol) * 100
    f = str(f) + '%'
    
    Final_list = {'items': liked_object_id, 'text_id': text_id, 'emoji': emojicount, 'tags': tagscount, 'links': linkcount, 'words': wordcount, 'count': User_list_of_responses[0],
                    'list of users': User_list_of_responses[1], 'likes_all': lol, 'likes_count (<21)': list_of_likes, 'likes_count (>31)': list_of_likes2, 'date': date_list[t], 'cnt_comm': cnt_comm_list[t], 'photo_ids': photo_ids_list[t], 'theme': text_analyze['Тематика'],
                    'headers_cnt': text_analyze['Структура текста']['Количество заголовков'], 'parts_cnt': text_analyze['Структура текста']['Количество абзацев'], 'tone': text_analyze['Язык и стиль']['Тональность'],
                    'difficult': text_analyze['Язык и стиль']['Сложность текста'], 'emote': text_analyze['Язык и стиль']['Эмоциональная окраска'], 'medium_photo_text': medium_photo_text, 'medium_photo_colorfulness': medium_photo_colorfulness}
    User_lists_collection.append(Final_list)
    t += 1
    df = pd.DataFrame(User_lists_collection)
    df.to_csv(f'file{t}.csv')

# Анализ данных

In [ ]:
files_to_merge = ['vtb.csv', 'tink.csv', 'sber.csv', 'file250.csv']
deli = 0
for i in files_to_merge:
    df1 = pd.read_csv(i)
    for j in range(len(df1['likes_all'])):
        if i == 'vtb.csv':
            deli = 0.65
        elif i == 'tink.csv':
            deli = 1.3
        elif i == 'sber.csv':
            deli = 3.37
        elif i == 'file250.csv':
            deli = 0.43 
        df.loc[df['items'] == df1['items'][j], 'likes'] = df1['likes_all'][j] / deli

In [ ]:
def round_to_nearest_hour(time_str):
    hours, minutes, _ = map(int, time_str.split(':'))
    # Увеличиваем час, если минуты >= 30
    if minutes >= 30:
        hours += 1
    return hours

# Применение функции к колонке
df['date'] = pd.to_datetime(df['date'], unit='s').dt.strftime('%H:%M:%S')
df['hour'] = df['date'].apply(round_to_nearest_hour)

### Обработка эмоций

In [ ]:
for i in range(len(df['emote'])):
    try:
        promt = f'Это суждение об эмоции {df['emote'][i]}. Ты должен написать "0", если эмоция нейтральная, "1", если эмоция радостная/положительная, "-1", если эмоция грустная/отрицательная. Ответь только число'
        ans = llama_request(promt)
        df['emote'][i] = ans
    except:
        pass

In [ ]:
print(df['emote'].unique())

In [ ]:
df.to_csv('after_emote_result.csv', index = False)

In [ ]:
print(df['emote'].unique())
for i in range(len(df['emote'])):
    if df['emote'][i] == '-1.':
        df['emote'][i] = -1
    elif df['emote'][i] == 'Нет':
        df['emote'][i] = 0
    elif df['emote'][i] == '0.':
        df['emote'][i] = 0
    elif df['emote'][i] == 'Я не могу ответить на этот вопрос в соответствии с правилами и политикой платформы. Я могу помочь вам с чем-то другим?':
        df['emote'][i] = 0
    elif df['emote'][i] == 'Используйте функцию анализа эмоций. Тон - 0 (нейтральный)':
        df['emote'][i] = 0
    elif df['emote'][i] == 'Радостная.':
        df['emote'][i] = 1
    elif df['emote'][i] == '1.':
        df['emote'][i] = 1
    elif df['emote'][i] == 'Ответ:  1.':
        df['emote'][i] = 1

## Выбросы

### Боксплоты для выбросов

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Стиль графика 'dark_background'
plt.style.use('default')

# Функция для построения boxplot с кастомными настройками
def plot_boxplots(df, columns, rows=2, cols=3, figsize=(18, 12)):
    fig, axes = plt.subplots(rows, cols, figsize=figsize)
    axes = axes.flatten()

    for i, column in enumerate(columns):
        # Создание boxplot для каждой колонки
        sns.boxplot(data=df, x=column, ax=axes[i], color='#8A2BE2')

        # Настройка графика
        axes[i].set_title(f"Boxplot для {column}", fontsize=14, color='black')
        axes[i].set_xlabel(column, fontsize=12, color='black')
        axes[i].tick_params(axis='x', colors='black', labelsize=10)
        axes[i].tick_params(axis='y', colors='black', labelsize=10)

        # Убираем рамку вокруг графиков
        '''
        axes[i].spines['top'].set_visible(False)
        axes[i].spines['right'].set_visible(False)
        axes[i].spines['left'].set_visible(False)
        axes[i].spines['bottom'].set_visible(False)
        '''
    
    # Убираем пустые графики, если колонок меньше чем количество ячеек
    if len(columns) < rows * cols:
        for j in range(len(columns), rows * cols):
            fig.delaxes(axes[j])

    # Настройка интервалов между графиками
    plt.tight_layout()

    # Сохранение графика
    plt.savefig('boxplots_grid.png', bbox_inches='tight', dpi=300)
    plt.show()

# Пример использования
columns_to_clean = ['words', 'likes_all']  # Здесь твои колонки
plot_boxplots(df, columns_to_clean)


### Чистка выбросов

In [ ]:
#Чистка выбросов и переименование
def remove_outliers_IQR(df, for_clean):
    df_cleaned = df.copy()
    for column in for_clean:
        original_count = df_cleaned.shape[0]
        
        Q1 = np.percentile(df_cleaned[column], 25)
        Q3 = np.percentile(df_cleaned[column], 75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        df_cleaned = df_cleaned[(df_cleaned[column] >= lower_bound) & (df_cleaned[column] <= upper_bound)]
        
        # Количество убранных значений
        cleaned_count = df_cleaned.shape[0]
        removed_counts = original_count - cleaned_count
        print(column , removed_counts)

    return df_cleaned


print(len(df))
start_len = len(df)
for_clean = ['words', 'likes_all']
df = remove_outliers_IQR(df, for_clean)
print(len(df))
print(f'Cleaned: {start_len - len(df)}')

## Таблицы распределения

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Указание стиля 'dark_background'
num_zones = 10  # количество зон
bins = np.linspace(df['words'].min(), df['words'].max(), num_zones + 1)

# Добавление зоны в DataFrame
df['zone'] = pd.cut(df['words'], bins, labels=False)

# Средние значения в каждой зоне
avg_points = df.groupby('zone').agg({'words': 'mean', 'likes_all': 'mean'}).reset_index()

# Указание стиля 'dark_background'
plt.style.use('default')

# Построение scatter plot
plt.figure(figsize=(30, 18))
plt.scatter(
    x=df['words'],
    y=df['likes_all'],
    color='#8A2BE2',  # насыщенный фиолетовый
    edgecolor='black'  # чёрная обводка
)

# Добавление средних точек
plt.scatter(
    x=avg_points['words'],
    y=avg_points['likes_all'],
    color='#8A2BE2',
    s=200,  # размер точек
    label='Среднее значение для промежутка'
)

# Построение кусочной функции
plt.plot(
    avg_points['words'],
    avg_points['likes_all'],
    color='#5781ff',
    linestyle='-',  # сплошная линия
    linewidth=4,
    label='Функция средних значений'
)

# Настройки графика
plt.title('Распределение лайков и слов', fontsize=24, color='black')
plt.xlabel('Слова', fontsize=20, color='black')
plt.ylabel('Лайки', fontsize=20, color='black')

# Легенда
plt.legend(fontsize=18, loc='upper right', frameon=False)

# Показ графика
plt.show()

## Таблица корреляций

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('default')
# Ваши данные
columns_for_corr = ['emoji', 'tags', 'links', 'words', 'likes_all', 'likes_count (<21)', 'likes_count (>31)', 'theme', 'medium_photo_text', 'medium_photo_colorfulness', 
                    'difficult', 'date', 'emote']
df_2 = df.loc[:, columns_for_corr].copy()

# Кастомная цветовая палитра
gradient_colors = [(1, 193/255, 136/255),  # FCC188
                   (112/255, 66/255, 210/255),  # 7042D2
                   (46/255, 0, 142/255)]  # 2E008E
cmap = LinearSegmentedColormap.from_list("custom_gradient", gradient_colors)

# Создание тепловой карты корреляций
plt.figure(figsize=(14, 12))
heatmap = sns.heatmap(df_2.select_dtypes(include=['number']).corr(method='spearman'), 
                        annot=True, 
                        cmap=cmap,  # Кастомная палитра
                        vmin=-1, 
                        vmax=1, 
                        fmt=".2f", 
                        linewidths=0.5,
                        annot_kws={"size": 18},  # Увеличение шрифта аннотаций
                        xticklabels=True,  # Метки по оси X
                        yticklabels=True)  # Метки по оси Y

# Заголовок и оси с увеличенным шрифтом
heatmap.tick_params(axis='both', which='major', labelsize=14, colors='black')

plt.show()
#Основное
#sber
#emoji - likes <21 и сравнение likes >31
#words - likes <21 и сравнение likes >31
#match_tv
#emoji - likes <21 и сравнение likes >31
#words - likes <21 и сравнение likes >31


In [ ]:
from scipy.stats import pearsonr, spearmanr
import pandas as pd

# Колонки для анализа
columns_for_corr = ['emoji', 'tags', 'links', 'words', 'likes_all',
                    'theme', 'medium_photo_text', 'medium_photo_colorfulness', 
                    'difficult', 'date', 'emote']

# Выбираем только числовые колонки
df_2 = df.loc[:, columns_for_corr].copy()

# Целевая колонка
target_column = 'likes_all'
cnt = 0

results = []
for column in columns_for_corr:
    if column != target_column:
        r, p_value = spearmanr(df_2[column], df_2[target_column])
        results.append({'Фактор': column, 'Коэффициент корреляции (r)': r, 'p-value': p_value})
    
results_df = pd.DataFrame(results)
results_df = results_df.round(7)

print(results_df)

### Тест Шапиро-Уилка

In [ ]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

# Генерация случайных данных (например, нормально распределенные)e=100)

# Проведение теста Шапиро-Уилка

# Вывод результатов

# Колонки для анализа
columns_for_corr = ['likes_all']

# Выбираем только числовые колонки
df_2 = df.loc[:, columns_for_corr].copy()

# Целевая колонка
# Расчет корреляции и p-value для каждой колонки
results = []
for column in columns_for_corr:
        print(column)
        statistic, p_value = stats.shapiro(df_2[column])
        print(f"Статистика теста: {statistic}")
        print(f"P-значение: {p_value}")

        # Интерпретация результата
        alpha = 0.05
        if p_value > alpha:
            print("Нет оснований отвергать нулевую гипотезу: данные нормально распределены.")
        else:
            print("Отвергаем нулевую гипотезу: данные не нормально распределены.")
    
# Сортируем по абсолютному значению коэффициента корреляции для удобства анализа
#results_df = results_df.sort_values(by='Коэффициент корреляции (r)', key=abs, ascending=False

### Разбиение на равные группы

In [ ]:
import pandas as pd

# Функция для вычисления границ групп
def calculate_bins(df):
    bin_info = {}
    for column in df.select_dtypes(include='number').columns:
        unique_values = df[column].nunique()  # Проверка уникальных значений
        if unique_values > 1:  # Если в колонке больше одного уникального значения
            try:
                bins = pd.qcut(df[column], q=3, duplicates='drop', retbins=True)[1]
                bin_info[column] = bins
            except ValueError:
                bin_info[column] = "Unable to split (insufficient variability)"
        else:
            bin_info[column] = "Single unique value"
    return bin_info

# Пример данных

# Вывод границ для каждой числовой колонки
bins = calculate_bins(df)
for col, bin_edges in bins.items():
    print(f"Column: {col}, Bins: {bin_edges}")


## Критерий Крускала-Уолеса

In [ ]:
import pandas as pd
from scipy.stats import kruskal

# Загружаем данные из файла
#data = df.copy()
groups = ['emoji_group', 'links_group', 'tags_group', 'parts_group', 'time_group', 'words_group', 'colorfulness_group', 'difficult_group', 'medium_photo_text_bins']

# Приведём данные к корректным типам, если потребуется
data['emote'] = data['emote'].astype(int)  # Убедимся, что emote числовой

# Разбивка переменной emoji на группы (Low, Medium, High)
emoji_bins = [-1, 1, 2, float('inf')]
emoji_labels = ['0', '0.5', '1']
data['emoji_group'] = pd.cut(data['emoji'], bins=emoji_bins, labels=emoji_labels)

links_bins = [-1, 3, 6, float('inf')]
links_labels = ['0', '0.5', '1']
data['links_group'] = pd.cut(data['links'], bins=links_bins, labels=links_labels)


tags_bins = [-1, 1, 2, float('inf')]
tags_labels = ['0', '0.5', '1']
data['tags_group'] = pd.cut(data['tags'], bins=tags_bins, labels=tags_labels)

parts_bins = [-1, 3, 4, float('inf')]
parts_labels = ['0', '0.5', '1']
data['parts_group'] = pd.cut(data['parts_cnt'], bins=parts_bins, labels=parts_labels)

time_bins = [4, 10, 13, float('inf')]
time_labels = ['0', '0.5', '1']
data['time_group'] = pd.cut(data['hour'], bins=time_bins, labels=time_labels)

words_bins = [-1, 58, 87, float('inf')]
words_labels = ['0', '0.5', '1']
data['words_group'] = pd.cut(data['words'], bins=words_bins, labels=words_labels)

colorfulness_bins = [-1, 2, 7, float('inf')]
colorfulness_labels = ['0', '0.5', '1']
data['colorfulness_group'] = pd.cut(data['medium_photo_colorfulness'], bins=colorfulness_bins, labels=colorfulness_labels)

difficult_bins = [1, 5, 7, float('inf')]
difficult_labels = ['0', '0.5', '1']
data['difficult_group'] = pd.cut(data['difficult'], bins=difficult_bins, labels=difficult_labels)

text_bins = [0.0, 0.6, 0.9, 1.0]
text_labels = ['0', '0.5', '1']  # Названия групп для удобства
data['medium_photo_text_bins'] = pd.cut(df['medium_photo_text'], bins=text_bins, labels=text_labels)


# Проведение теста Крускала-Уоллиса для likes_all по emoji_group


kruskal_emoji = kruskal(
    *[data.loc[data['emoji_group'] == group, 'likes_all'] for group in emoji_labels]
)

kruskal_emote = kruskal(
    *[data.loc[data['emote'] == emote, 'likes_all'] for emote in [-1, 0, 1]]
)

kruskal_time = kruskal(
    *[data.loc[data['time_group'] == hour, 'likes_all'] for hour in time_labels]
)
# Проведение теста Крускала-Уоллиса для likes_all по emoji_group

kruskal_word = kruskal(
    *[data.loc[data['words_group'] == group, 'likes_all'] for group in words_labels]
)
kruskal_colorfulness = kruskal(
    *[data.loc[data['colorfulness_group'] == group, 'likes_all'] for group in colorfulness_labels]
)
kruskal_difficult = kruskal(
    *[data.loc[data['difficult_group'] == group, 'likes_all'] for group in difficult_labels]
)
kruskal_text = kruskal(
    *[data.loc[data['medium_photo_text_bins'] == group, 'likes_all'] for group in text_labels]
)
kruskal_parts = kruskal(
    *[data.loc[data['parts_group'] == group, 'likes_all'] for group in parts_labels]
)
kruskal_links = kruskal(
    *[data.loc[data['links_group'] == group, 'likes_all'] for group in links_labels]
)

kruskal_emoji, kruskal_emote, kruskal_time, kruskal_word, kruskal_colorfulness, kruskal_difficult, kruskal_text, kruskal_parts, kruskal_links


# Модель машинного обучения

### Random Forest

In [ ]:
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import joblib
# Кластеризация
data = df.copy()
kmeans = KMeans(n_clusters=3, random_state=42)
data['cluster'] = kmeans.fit_predict(data[['emoji', 'words', 'medium_photo_text', 'medium_photo_colorfulness', 'hour', 'parts_cnt', 'emote', 'difficult']])

# Делим данные на обучающую и тестовую выборки
X = data[['emoji', 'words', 'medium_photo_text', 'hour', 'parts_cnt', 'emote', 'medium_photo_colorfulness', 'difficult', 'cluster']]
y = data['likes_all']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение модели
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Предсказания
y_pred = model.predict(X_test)
joblib.dump(model, 'random_forest_model.pkl')
print("Модель сохранена в 'random_forest_model.pkl'")
# Оценка
mse = mean_squared_error(y_test, y_pred)
print(f"Среднеквадратическая ошибка: {mse}")


### Поиск доп. параметров

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
import seaborn as sns
import matplotlib.pyplot as plt

# Пример данных (замените на свои)

# Создание совокупных факторов
data['few_words_and_no_emoji'] = (data['words'] < 60.0) & (data['emoji'] == 0) # Малое количество слов и эмодзи
data['few_words_and_few_tags'] = (data['words'] < 60.0) & (data['emoji'] > 0) # Малое количество слов и эмодзи
data['emoji_and_link'] = (data['emoji'] > 0) & (df['links'] > 0)  # Эмодзи и ссылка
data['emoji_and_tags'] = (data['emoji'] > 0) & (df['links'] > 0)  # Эмодзи и ссылка
data['low_difficulty_few_words'] = (data['difficult'] <= 5) & (data['words'] < 60.0)  # Сложный и длинный текст
data['few_words_photo_text'] = (data['words'] < 60.0) & (data['medium_photo_text'] > 0)  # Малое количество слов и текст на фото
data['many_words_big_difficult'] = (data['words'] > 60.0) & (data['difficult'] > 6)  # Эмодзи и низкая сложность текста

# Преобразование булевых значений в числовые

# Список факторов
factors = ['many_words_big_difficult','few_words_photo_text', 'low_difficulty_few_words', 'emoji_and_tags', 'emoji_and_link', 'few_words_and_few_tags', 'few_words_and_no_emoji']

# Корреляционный анализ
correlation_results = {}
for factor in factors:
    corr, p_value = spearmanr(data[factor], data['likes_all'])
    correlation_results[factor] = {'correlation': corr, 'p_value': p_value}

# Вывод результатов корреляции
for factor, result in correlation_results.items():
    print(f"Factor: {factor}")
    print(f"  Spearman correlation: {result['correlation']:.7f}")
    print(f"  P-value: {result['p_value']:.7f}")
    print()

# Визуализация факторов

### Итоговый XGBoost

In [ ]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
#data = data.drop(columns = ['Unnamed: 0.1', 'Unnamed: 0', 'items', 'count', 'list of users', 'text_id',  'likes_count (<21)', 'likes_count (>31)', 'cnt_comm', 'photo_ids', 'theme', 'tone', 'medium_photo_text_bins', 'date', 'likes', 'zone'])

data['cluster'] = kmeans.fit_predict(data[['emoji', 'words', 'medium_photo_text', 'medium_photo_colorfulness', 'hour', 'parts_cnt', 'emote', 'difficult']])

param_grid = {
    "max_depth": [3,4, 5, 6, 7],
    "learning_rate": [0.01, 0.05, 0.1, 0.2],
    "subsample": [0.6, 0.7, 0.8, 0.9, 1.0],
    "colsample_bytree": [0.4, 0.5, 0.6, 0.8, 1.0],
    "n_estimators": [100, 200, 300, 400, 500]
}

X = data.drop(columns=['likes_all'])  # Убираем колонку с целевой переменной
y = data['likes_all']  # Целевая переменная

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features = 1000)
text_features = vectorizer.fit_transform(df['text_id']).toarray()
import numpy as np
X = np.hstack((X, text_features))
xgb = XGBRegressor(objective='reg:squarederror')

# Разделяем данные на тренировочные и тестовые выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    scoring="neg_mean_squared_error",
    cv=3,
    verbose=1,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)
print(f"Лучшие параметры: {grid_search.best_params_}")
print(f"Лучший AUC: {-grid_search.best_score_}")


In [ ]:
import joblib

# Сохранение обученной модели
joblib.dump(grid_search.best_estimator_, "best_xgb_model.pkl")

# Загрузка сохранённой модели
loaded_model = joblib.load("best_xgb_model.pkl")

## Регрессия

In [ ]:
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

df['hour'] = pd.to_datetime(df['date'], format='%H:%M:%S').dt.hour

# One-Hot Encoding для 'emote'

# Преобразование всех булевых значений в числовые

# Выбор независимых переменных
X = df[['emoji', 'tags', 'links', 'words', 'medium_photo_text', 'medium_photo_colorfulness', 
                    'difficult', 'hour']]

# Целевая переменная
y = df['likes_all']

# Добавление константы для модели
X = sm.add_constant(X)

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Построение модели
model = sm.OLS(y_train, X_train).fit()

# Вывод результатов модели
print(model.summary())